In [2]:
import os
import pandas as pd

In [3]:
# Set the path to the target directory
base_path = "C:/Users/Ally/Desktop/sheng/data/6/Stress_dataset"

# Get a list of all subdirectories in "1"
subject_dirs = [name for name in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, name))]

# Print each subdirectory name
for subdir in subject_dirs:
    print(subdir)
    
chunks_all = []

15
5C
6B
6D
7A
7E
83
8B
94
BG
CE
DF
E4
EG
F5


In [5]:
for j in range(len(subject_dirs)):
    subject_dirs[j]
    
    term_dirs = os.listdir(os.path.join(base_path, subject_dirs[j]))
    for k in range(len(term_dirs)):

        BVP_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "BVP.csv")
        EDA_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "EDA.csv")
        TEMP_path = os.path.join(base_path, subject_dirs[j], term_dirs[k], "TEMP.csv")

        # print(BVP_path)
        # print(EDA_path)
        # print(TEMP_path)
        
        # Read the CSV file
        df_BVP = pd.read_csv(BVP_path, header=None)
        df_EDA = pd.read_csv(EDA_path, header=None)
        df_TEMP = pd.read_csv(TEMP_path, header=None)

        '''
        if df_BVP.iloc[0,0] == df_EDA.iloc[0,0] and df_BVP.iloc[0,0] == df_TEMP.iloc[0,0]:
            print(1)
        else:
            print(0)
        '''

        df_BVP = df_BVP.iloc[2:].reset_index(drop=True)
        df_EDA = df_EDA.iloc[2:].reset_index(drop=True)
        df_TEMP = df_TEMP.iloc[2:].reset_index(drop=True)

        # downsample BVP to 32 hz
        df_BVP = df_BVP.iloc[::2].reset_index(drop=True)
        df_EDA = df_EDA.loc[df_EDA.index.repeat(8)].reset_index(drop=True)
        df_TEMP = df_TEMP.loc[df_TEMP.index.repeat(8)].reset_index(drop=True)

        # combine all three df into one data frame
        # Step 1: Find the minimum number of rows
        min_rows = min(len(df_BVP), len(df_EDA), len(df_TEMP))

        # Step 2: Cut each dataframe to the minimum number of rows
        df_BVP_cut = df_BVP.iloc[:min_rows].reset_index(drop=True)
        df_EDA_cut = df_EDA.iloc[:min_rows].reset_index(drop=True)
        df_TEMP_cut = df_TEMP.iloc[:min_rows].reset_index(drop=True)

        # Step 3: combine columns
        df= pd.concat([df_BVP_cut, df_EDA_cut, df_TEMP_cut], axis=1)
    
        # assign column names
        df.columns = ['BVP', 'EDA', 'TEMP']

        # set index as a column named "time"
        df = df.reset_index().rename(columns={'index': 'time'})
        df['subject'] = subject_dirs[j]

        # chunk df into chunks of 4 seconds with 1 second overlap
        # Parameters
        fs = 32              # Sampling frequency (Hz)
        chunk_duration = 4   # seconds
        overlap_duration = 1 # seconds

        chunk_size = chunk_duration * fs
        overlap_size = overlap_duration * fs
        step_size = chunk_size - overlap_size  # How many rows to move forward for each new chunk

        # Store all chunks
        chunks = []

        # Go through the DataFrame
        for start in range(0, len(df) - chunk_size + 1, step_size):
            chunk = df.iloc[start : start + chunk_size].reset_index(drop=True)
            chunks.append(chunk)

        # Now `chunks` is a list of DataFrames, each 128 rows, overlapping by 32 rows

        chunks_all.append(chunks)

In [14]:
chunks_all[300]

[     time    BVP       EDA   TEMP subject
 0       0  -0.00  0.000000  30.73      94
 1       1  -0.00  0.000000  30.73      94
 2       2  -0.00  0.000000  30.73      94
 3       3  -0.00  0.000000  30.73      94
 4       4  -0.00  0.000000  30.73      94
 ..    ...    ...       ...    ...     ...
 123   123 -83.76  0.065348  30.71      94
 124   124 -37.22  0.065348  30.71      94
 125   125  14.51  0.065348  30.71      94
 126   126  22.70  0.065348  30.71      94
 127   127  -0.38  0.065348  30.71      94
 
 [128 rows x 5 columns],
      time     BVP       EDA   TEMP subject
 0      96    1.72  0.000000  30.71      94
 1      97    2.12  0.000000  30.71      94
 2      98    2.20  0.000000  30.71      94
 3      99    2.52  0.000000  30.71      94
 4     100    3.47  0.000000  30.71      94
 ..    ...     ...       ...    ...     ...
 123   219  103.77  0.139665  30.55      94
 124   220  159.06  0.139665  30.55      94
 125   221  170.70  0.139665  30.55      94
 126   222  120.0

In [9]:
len(chunks_all[1])

702

In [8]:
len(chunks_all)

609

In [15]:
for i, chunk in enumerate(chunks_all):
    pd.concat(chunk, axis = 0).reset_index(drop = True).to_csv(f'chunk_{i}.csv', index = False)